# UFC-Fight historical data from 1993 to 2021

In [1]:
import numpy as np
import pandas as pd
import altair as alt

## Introdução

O dataset utilizado possui informações de dois jogadores em uma luta de UFC, onde para cada jogador é armazenada a sua
idade, o seu peso em libras (*pounds*), a sua altura e alcançe de ataque em centímetros, a postura utilizada na luta 
(podendo ser *Open Stance*, *Orthodox*, *Southpaw* e *Switch*) e o número de vitórias, derrotas e tempo de luta anteriores
à luta. Também há informações sobre a luta, como o dia, se é uma final e o seu ganhador.

Um jogador pode ser da cor vermelha ou azul.

In [2]:
df = pd.read_csv(
    'data.csv',
    usecols=('B_age', 'R_age', 'B_Weight_lbs', 'R_Weight_lbs', 'B_Height_cms', 'R_Height_cms', 'B_Reach_cms', 'R_Reach_cms', 'B_Stance', 'R_Stance', 'Winner', 'title_bout', 'date', 'B_wins', 'R_wins', 'B_losses', 'R_losses', 'R_total_time_fought(seconds)', 'B_total_time_fought(seconds)', 'weight_class'),
    parse_dates=['date']
).dropna().reset_index()

In [3]:
df

,index,date,Winner,title_bout,weight_class,B_total_time_fought(seconds),B_wins,B_losses,B_Stance,B_Height_cms,...,B_Weight_lbs,R_total_time_fought(seconds),R_wins,R_losses,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,0,2021-03-20,Red,False,Bantamweight,531.50000,1,1,Orthodox,165.10,...,135.0,166.000000,1,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,1,2021-03-20,Red,False,Middleweight,577.50000,2,0,Orthodox,187.96,...,205.0,747.343750,4,2,Orthodox,182.88,187.96,185.0,32.0,28.0
2,4,2021-03-20,Blue,False,WomenBantamweight,764.00000,4,1,Orthodox,180.34,...,135.0,874.380859,5,6,Orthodox,167.64,172.72,135.0,29.0,43.0
3,5,2021-03-20,Blue,False,Lightweight,717.87500,4,0,Switch,177.80,...,145.0,649.304688,7,1,Orthodox,182.88,190.50,155.0,27.0,41.0
4,6,2021-03-20,Blue,False,Welterweight,809.53125,4,6,Orthodox,180.34,...,170.0,445.000000,4,1,Orthodox,182.88,180.34,170.0,35.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3909,5759,2001-02-23,Red,True,LightHeavyweight,312.75000,3,0,Orthodox,182.88,...,185.0,645.578125,5,2,Orthodox,190.50,187.96,205.0,30.0,26.0
3910,5767,2000-12-16,Red,False,Middleweight,600.00000,1,0,Orthodox,175.26,...,247.0,476.500000,2,1,Orthodox,187.96,193.04,205.0,29.0,30.0
3911,5799,2000-04-14,Red,True,LightHeavyweight,108.50000,1,1,Orthodox,180.34,...,205.0,858.312500,3,2,Orthodox,190.50,187.96,205.0,23.0,25.0
3912,5840,1999-03-05,Red,False,Middleweight,720.00000,1,0,Orthodox,187.96,...,205.0,597.500000,0,2,Orthodox,185.42,187.96,185.0,29.0,23.0


## Visualização geral

### Posturas usadas em combate

In [4]:
base = alt.Chart(df).transform_aggregate(
    Usos='count()',
    groupby=['R_Stance', 'B_Stance']
).encode(
    alt.X('R_Stance:O', scale=alt.Scale(paddingInner=0)),
    alt.Y('B_Stance:O', scale=alt.Scale(paddingInner=0)),
).properties(
    width=400,
    height=300,
)

heatmap = base.mark_rect().encode(
    color=alt.Color('Usos:Q',
        scale=alt.Scale(scheme='viridis'),
        legend=alt.Legend(direction='vertical')
    )
)

text = base.mark_text(baseline='middle').encode(
    text='Usos:Q',
    color=alt.condition(
        alt.datum.Usos > 1000,
        alt.value('black'),
        alt.value('white')
    )
)

heatmap + text

alt.LayerChart(...)

In [5]:
import vega_datasets
vega_datasets.data.barley()

,yield,variety,year,site
0,27.00000,Manchuria,1931,University Farm
1,48.86667,Manchuria,1931,Waseca
2,27.43334,Manchuria,1931,Morris
3,39.93333,Manchuria,1931,Crookston
4,32.96667,Manchuria,1931,Grand Rapids
...,...,...,...,...
115,58.16667,Wisconsin No. 38,1932,Waseca
116,47.16667,Wisconsin No. 38,1932,Morris
117,35.90000,Wisconsin No. 38,1932,Crookston
118,20.66667,Wisconsin No. 38,1932,Grand Rapids


### Número de lutas por peso

In [6]:
df_ = df.groupby([df.date.dt.year, 'weight_class']).count()
df_ = pd.DataFrame([[year, c, r['index']] for (year, c), r in df_.iterrows()], columns=['Ano', 'Categoria', 'Lutas'])

alt.Chart(df_).mark_bar().encode(
    x=alt.X('sum(Lutas)', stack="normalize"),
    y='Ano',
    color='Categoria'
).properties(
    width=600,
    height=500,
)

alt.Chart(...)

## Análise

- Qual a postura mais utilizada pelos lutadores?

Segundo o gráfico **Posturas utilizadas em combate**, a Orthodox, com 2235 lutas onde os dois jogadores a utilizavam.

- A partir de que ano foi incluído a modalidade feminina no UFC?

Segundo o gráfico **Número de lutas por peso**, a partir de 2013 as categorias Galo (*Bantamweight)*, de 2014 
a categoria Palha (*Strawweight*) e de 2017 a categoria Pena e Mosca (*Featherweight* e *Flyweight*).
